In [1]:
import pandas as pd
import numpy as np

In [2]:
import math, string, sys, fileinput

def range_bytes (): return range(256)
def range_printable(): return (ord(c) for c in string.printable)
def H(data, iterator=range_bytes):
    if not data:
        return 0
    entropy = 0
    for x in iterator():
        p_x = float(data.count(chr(x)))/len(data)
        if p_x > 0:
            entropy += - p_x*math.log(p_x, 2)
    return entropy



for str_ in ['gargleblaster', 'tripleee', 'magnus', 'lkjasdlk',
               'aaaaaaaa', 'sadfasdfasdf', '7&wS/p(']:
    print ("%s: %f" % (str_, H(str_, range_printable)))



gargleblaster: 2.931209
tripleee: 2.405639
magnus: 2.584963
lkjasdlk: 2.500000
aaaaaaaa: 0.000000
sadfasdfasdf: 2.000000
7&wS/p(: 2.807355


In [3]:
df_train = pd.read_csv('train.csv').dropna()

In [4]:
df_train.head()

,Password,Times
0,631XniVx2lS5I,2
1,LEGIT747,1
2,742364es,1
3,3846696477,1
4,laurahop,2


In [5]:
df_train['Times'].value_counts(normalize=True).head(20)*100

1     85.442855
2      7.448042
3      2.323525
4      1.149080
5      0.672119
6      0.448682
7      0.328436
8      0.254535
9      0.198362
10     0.163194
11     0.133590
12     0.116055
13     0.100566
14     0.085873
15     0.074937
16     0.065013
17     0.061496
18     0.053523
19     0.048224
20     0.045454
Name: Times, dtype: float64

In [6]:
df_test = pd.read_csv('Xtest.csv')

In [7]:
df_test.head()

,Id,Password
0,0,ThaisCunha
1,1,697775113
2,2,922a16922a
3,3,andy74
4,4,joemack


In [8]:
df_train = df_train[df_train['Times']<5]
X_test = df_test['Password']
X_train = df_train['Password']
y_train= df_train['Times']

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_log_error
from string import punctuation
#from catboost import CatBoostRegressor 

from sklearn.ensemble import RandomForestRegressor

/Users/mikhail_p/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
df_train['length'] = df_train['Password'].apply(lambda x: len(x))
df_train['has_lower'] = df_train['Password'].apply(lambda x: any(char.islower() for char in x))
df_train['has_digits'] = df_train['Password'].apply(lambda x: any(char.isdigit() for char in x))
df_train['has_upper'] = df_train['Password'].apply(lambda x: any(char.isupper() for char in x))
df_train['has_special'] = df_train['Password'].apply(lambda x: any(char in set(punctuation) for char in x))
df_train['H']= df_train['Password'].apply(H)
df_train['isalpha'] = df_train['Password'].str.isalpha()
df_train['isdigit'] = df_train['Password'].str.isdigit()
df_train['vowel_num'] = df_train['Password'].apply(lambda x: sum(char in 'aeiou' for char in x.lower()))
df_train['consonant_num'] = df_train['Password'].apply(lambda x: sum(char in 'bcdfghjklmnpqrstvwxyz' for char in x.lower()))
df_train['digit_num'] = df_train['Password'].apply(lambda x: sum(char in '1234567890' for char in x.lower()))
df_train['vowel_percentage'] = df_train['vowel_num'] / df_train['length']
df_train['consonant_percentage'] = df_train['consonant_num'] / df_train['length']
df_train['letter_percentage'] = (df_train['vowel_num'] + df_train['consonant_num']) / df_train['length']
df_train['digit_percentage'] = df_train['digit_num'] / df_train['length']



In [11]:
df_train.columns

Index(['Password', 'Times', 'length', 'has_lower', 'has_digits', 'has_upper',
       'has_special', 'H', 'isalpha', 'isdigit', 'vowel_num', 'consonant_num',
       'digit_num', 'vowel_percentage', 'consonant_percentage',
       'letter_percentage', 'digit_percentage'],
      dtype='object')

In [12]:
X_train = df_train[['length', 'has_lower', 'has_digits', 'has_upper',
       'has_special', 'H', 'isalpha', 'isdigit', 'vowel_num', 'consonant_num',
       'digit_num', 'vowel_percentage', 'consonant_percentage',
       'letter_percentage', 'digit_percentage']]

In [13]:
y_pred = cross_val_predict(RandomForestRegressor(random_state=42), X_train, y_train, verbose=30,n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.8min finished


In [14]:
np.sqrt(np.mean((np.log(y_pred+1) - np.log(y_train.astype(float)+1))**2))

0.17177503169938305

In [15]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
X_test[X_test.isnull()] = ''

/Users/mikhail_p/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_test['length'] = df_test['Password'].apply(lambda x: len(x))
df_test['has_lower'] = df_test['Password'].apply(lambda x: any(char.islower() for char in x))
df_test['has_digits'] = df_test['Password'].apply(lambda x: any(char.isdigit() for char in x))
df_test['has_upper'] = df_test['Password'].apply(lambda x: any(char.isupper() for char in x))
df_test['has_special'] = df_test['Password'].apply(lambda x: any(char in set(punctuation) for char in x))
df_test['H']= df_test['Password'].apply(H)
df_test['isalpha'] = df_test['Password'].str.isalpha()
df_test['isdigit'] = df_test['Password'].str.isdigit()
df_test['vowel_num'] = df_test['Password'].apply(lambda x: sum(char in 'aeiou' for char in x.lower()))
df_test['consonant_num'] = df_test['Password'].apply(lambda x: sum(char in 'bcdfghjklmnpqrstvwdxyz' for char in x.lower()))
df_test['digit_num'] = df_test['Password'].apply(lambda x: sum(char in '1234567890' for char in x.lower()))
df_test['vowel_percentage'] = df_test['vowel_num'] / df_test['length']
df_test['consonant_percentage'] = df_test['consonant_num'] / df_test['length']
df_test['letter_percentage'] = (df_test['vowel_num'] + df_test['consonant_num']) / df_test['length']
df_test['digit_percentage'] = df_test['digit_num'] / df_test['length']




In [18]:
X_test = df_test[['length', 'has_lower', 'has_digits', 'has_upper',
       'has_special', 'H', 'isalpha', 'isdigit', 'vowel_num', 'consonant_num',
       'digit_num', 'vowel_percentage', 'consonant_percentage',
       'letter_percentage', 'digit_percentage']]

In [19]:
X_test.head()

,length,has_lower,has_digits,has_upper,has_special,H,isalpha,isdigit,vowel_num,consonant_num,digit_num,vowel_percentage,consonant_percentage,letter_percentage,digit_percentage
0,10,True,False,True,False,2.921928,True,False,4,6,0,0.400000,0.600000,1.000000,0.000000
1,9,False,True,False,False,2.419382,False,True,0,0,9,0.000000,0.000000,0.000000,1.000000
2,10,True,True,False,False,2.121928,False,False,2,0,8,0.200000,0.000000,0.200000,0.800000
3,6,True,True,False,False,2.584963,False,False,1,3,2,0.166667,0.500000,0.666667,0.333333
4,7,True,False,False,False,2.807355,True,False,3,4,0,0.428571,0.571429,1.000000,0.000000


In [20]:
X_test = X_test.replace(np.nan, 0)

In [21]:
y_pred = rf.predict(X_test) 

In [22]:
df_res = pd.DataFrame({'Id':df_test['Id'], 'Times':y_pred}, columns=['Id', 'Times'])

In [24]:
#df_res['Times'] = np.round(df_res['Times'])

In [25]:
df_res.to_csv('rf_more_features_entopy_res.csv',sep=',',index=0)